# Building Effective Agents

This is based on [this article](https://www.anthropic.com/engineering/building-effective-agents) by [Anthropic](https://www.anthropic.com/). We will explore common patterns for building effective LLM agentic systems using pure Python around LLM APIs. In particular, we use the [Python SDK](https://github.com/openai/openai-python/tree/main) for the [OpenAI API](https://platform.openai.com/docs/api-reference/introduction). 

## OpenAI API client

First, we need to load the API key in the environmental variables. The client expects the variable name `OPENAI_API_KEY` which we load from the `.env` file. This is easy to implement:

In [1]:
import inspect
from notebooks.utils import load_dotenv
print(inspect.getsource(load_dotenv))

def load_dotenv(verbose=False):
    with open(".env") as f:
        for line in f.readlines():
            k, v = line.split("=")
            os.environ[k] = v.strip().strip('"')
            if verbose:
                print(f"Loaded env variable: {k}")



In [2]:
load_dotenv(verbose=True)

Loaded env variable: OPENAI_API_KEY


Then the API key is automatically read by the **client**:

In [3]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system", 
            "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "Tell me about the history of GPT in a single paragraph.",
        },
    ],
)

response_text = completion.choices[0].message.content

:::{.callout-note}
We can have **multiple completions** of the same prompt. This allows choosing between the responses.
For example, we can set `temperature=0.9` to get more varied outputs, so that choosing becomes nontrivial. 
By default only 1 completion by default, hence `[0]`.
:::

In [4]:
from pprint import pprint

pprint(response_text, compact=True)

('Generative Pre-trained Transformer (GPT) is a series of language models '
 'developed by OpenAI, beginning with the release of GPT in 2018. This model '
 'introduced the concept of leveraging unsupervised learning to pre-train a '
 'transformer-based architecture on a large corpus of text, which could then '
 'be fine-tuned for specific tasks. Successive iterations like GPT-2, released '
 'in 2019, showcased significant improvements in generating coherent and '
 'contextually relevant text, though its release was initially limited due to '
 'concerns over misuse. GPT-3, launched in 2020, expanded upon this by using '
 '175 billion parameters, offering even more nuanced and human-like text '
 'generation and understanding capabilities, leading to applications across '
 'creative writing, coding, and conversational agents. In 2023, OpenAI '
 "introduced GPT-4, which further enhanced the model's depth and versatility, "
 'strengthening its ability to process and generate complex languag

In [5]:
from pprint import pprint
pprint(response_text)

('Generative Pre-trained Transformer (GPT) is a series of language models '
 'developed by OpenAI, beginning with the release of GPT in 2018. This model '
 'introduced the concept of leveraging unsupervised learning to pre-train a '
 'transformer-based architecture on a large corpus of text, which could then '
 'be fine-tuned for specific tasks. Successive iterations like GPT-2, released '
 'in 2019, showcased significant improvements in generating coherent and '
 'contextually relevant text, though its release was initially limited due to '
 'concerns over misuse. GPT-3, launched in 2020, expanded upon this by using '
 '175 billion parameters, offering even more nuanced and human-like text '
 'generation and understanding capabilities, leading to applications across '
 'creative writing, coding, and conversational agents. In 2023, OpenAI '
 "introduced GPT-4, which further enhanced the model's depth and versatility, "
 'strengthening its ability to process and generate complex languag

Entire model response:

In [6]:
from pprint import pprint
pprint(completion.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': [],
                          'audio': None,
                          'content': 'Generative Pre-trained Transformer (GPT) '
                                     'is a series of language models developed '
                                     'by OpenAI, beginning with the release of '
                                     'GPT in 2018. This model introduced the '
                                     'concept of leveraging unsupervised '
                                     'learning to pre-train a '
                                     'transformer-based architecture on a '
                                     'large corpus of text, which could then '
                                     'be fine-tuned for specific tasks. '
                                     'Successive iterations like GPT-2, '
                                     'released in 201

:::{.callout-note}
We are using the **chat completions** API where an autoregressive process that's running under the hood. Here the prompt to be completed is:

```python
messages=[
    {"role": "system", "content": "You are a poetic but terse assistant."},   # prompt
    {"role": "user", "content": "What is the color of the sky?"}              # prompt
]
```

And the completion is given by the API's output:

```python
{
  "role": "assistant",
  "content": "The sky's color shifts from azure to amber, a canvas for sun's daily journey."
}
```

**Remark.** The generated response is statistically the most likely continuation of the prompt text sequence. 
:::

## Structured outputs

[Structured outputs](https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses) is a feature that ensures that a model generates responses that adhere to a supplied **schema** (e.g. a Pydantic model). As such, the output can then be parsed using the same Pydantic model. Structured outputs makes prompting significantly simpler: no more need for strongly worded prompts to achieve consistent formatting, no explicitly having to retry incorrectly formatted responses, or having invalid hallucinated values (can specify **enums**).

In [7]:
# https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

## Building block: Augmented LLM

This is the foundational building block of agentic systems. The **augmented LLM** is a language model enhanced with **retrieval**, **tools**, and **memory**. Current models, due to their reasoning and understanding capabilities, are able to effectively generate their own search queries, select appropriate tools, and determinine what information in necessary to retain or retrieve to solve problems or perform tasks. Augmentation is necessary for LLMs to reason about data outside of their training data, or data that is outdated relative to the training cutoff.

![**The augmented LLM.** A conceptual diagram of a core LLM service augmented with retrieval, tools, and memory capabilities.](../img/augmented-llm.png){#fig-augmentedllm}

### Tool calling

Also known as **function calling**. Function calling give models access to external tools and data that they can use to respond to prompts. Since LLMs *only* consume and generate text, they cannot actually execute functions. Instead, the main program listens to the LLM hallucinate and executes the commands based on that (@fig-brainvat).

![(**right**) LLM as brain in a vat that hallucinates outputs from information contained in inputs. It tells us *what* function to execute with *what* arguments. (**left**) The computer listens to the LLM and performs computation based on it.](../img/llm-brain-vat.png){#fig-brainvat width=80%}

To show tool calling, we find the weather in [Quisao](https://www.philatlas.com/luzon/r04a/rizal/pililla/quisao.html). We want the agent to call the following API:

In [8]:
import json
import requests

def get_weather(latitude, longitude):
    """
    Get current weather data for provided coordinates with units:
    temperature (celsius), wind speed (kph), & precipitation (mm).
    """
    response = requests.get((
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&"
        "current=temperature_2m,wind_speed_10m,relative_humidity_2m,precipitation,precipitation_probability"
    ))
    data = response.json()
    return data["current"]


get_weather(latitude=14.4779, longitude=121.3214)  # true coordinates

{'time': '2025-08-25T10:30',
 'interval': 900,
 'temperature_2m': 28.4,
 'wind_speed_10m': 9.4,
 'relative_humidity_2m': 83,
 'precipitation': 0.5,
 'precipitation_probability': 91}

**Tool definition.** For the LLM to understand a specific tool, we have to define a schema that informs the model of what the tool does and its expected (required and optional) arguments. The following is the function definition for `get_weather`:

In [9]:
tools = [
    {
        "type": "function", # <1>
        "function": {
            "name": "get_weather",  # <2>
            "description": "Get current weather data for provided coordinates with units: temperature (celsius), wind speed (kph), & precipitation (mm).",    # <3>
            "parameters": {
                "type": "object", # <4>
                "properties": { # <5>
                    "latitude": {
                        "type": "number"
                    },
                    "longitude": {
                        "type": "number"
                    },
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,  # <6>
            },
            "strict": True, # <7>
        },
    }
]

1. Should always be function.
2. Function's name (i.e. `get_weather`).
3. Usually just the docstring. Should describe when and how to use the function.
4. Parameters for LLMs are naturally JSON objects. Because `parameters` is defined by a JSON schema, you can leverage many of its rich features like property types, enums, descriptions, nested objects, and so on. For example, we can have: 
    ```
    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "unit of measure for temperature"}
    ```
5. List of arguments. Clearly the two arguments are required.
6. Part of JSON schema that determines whether extra fields are valid or not.
7. Not part of JSON schema, but OpenAI function calling option that *guides* the model to strictly follow the schema (i.e. not improvise). Setting `additionalProperties` to `False` and `strict` to `True` works together to ensure that the model generates the correct parameters schema.

In [10]:
system_prompt = "You are a helpful weather assistant."
user_prompt = "What's the weather like in Quisao, Pililla, Rizal right now?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

pprint(completion.choices[0].message.model_dump())

{'annotations': [],
 'audio': None,
 'content': None,
 'function_call': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"latitude":14.4629,"longitude":121.3706}',
                              'name': 'get_weather'},
                 'id': 'call_H2GV3HT3Kod05YksTqoUHla2',
                 'type': 'function'}]}


Impressive that it's able to get fairly accurate coordinates without using web search. Also notice that whenever we have `tools`, the model responds with only `tool_calls` as nonempty (e.g. `content` is empty). 
We will iterate over tool calls and process them separately. Each function call and their results are then logged as part of the sequence of **chat messages**. This explains why we defined `messages` outside of the API call unlike the usual setup.

In [11]:
def call_function(name, args):
    fn = {
        "get_weather": get_weather  
    }
    return fn[name](**args)


assistant_message = completion.choices[0].message
messages.append(assistant_message.model_dump())  # <1>

for tool_call in assistant_message.tool_calls:
    args = json.loads(tool_call.function.arguments)
    name = tool_call.function.name
    tool_output = call_function(name, args)
    messages.append({    # <2>
        "role": "tool", 
        "tool_call_id": tool_call.id, 
        "content": json.dumps(tool_output)
    })

1. The tool call is logged with role `assistant`.
2. Next, we log the result of the function call with role `tool`. 

In [12]:
pprint(messages)

[{'content': 'You are a helpful weather assistant.', 'role': 'system'},
 {'content': "What's the weather like in Quisao, Pililla, Rizal right now?",
  'role': 'user'},
 {'annotations': [],
  'audio': None,
  'content': None,
  'function_call': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"latitude":14.4629,"longitude":121.3706}',
                               'name': 'get_weather'},
                  'id': 'call_H2GV3HT3Kod05YksTqoUHla2',
                  'type': 'function'}]},
 {'content': '{"time": "2025-08-25T10:30", "interval": 900, "temperature_2m": '
             '26.0, "wind_speed_10m": 4.6, "relative_humidity_2m": 89, '
             '"precipitation": 0.5, "precipitation_probability": 92}',
  'role': 'tool',
  'tool_call_id': 'call_H2GV3HT3Kod05YksTqoUHla2'}]


:::{.callout-note}
Chat completion API calls have stateless single request-response cycles which gives the user straightforward control over the **message history**. This can be seen in the above example where we manually manage message history with tool calls declaration as well as actual function outputs.
:::

Next, we pass this thread to another API call (possibly to a different LLM) which will process the outputs of the tool calls along with earlier chat messages. To take advantage of structured outputs we again define a response format. Here we use Pydantic `Field` with a description that helps the LLM.

In [13]:
from pydantic import Field

class WeatherResponse(BaseModel):
    response: str = Field(description="A natural language response to the user's question.")
    temperature: float = Field(description="Current temperature in celsius for the given location.")


completion_weather = client.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,    # <!>
    response_format=WeatherResponse,
)

:::{.callout-caution}
The aggregator also needs access to tools for it to understand the context of each tool call!
:::

**Final output.** Note that even units (included in the `get_weather` docstring) were correctly identified:

In [14]:
parsed_weather = completion_weather.choices[0].message.parsed
print(parsed_weather.temperature)
pprint(parsed_weather.response)

26.0
('The weather in Quisao, Pililla, Rizal is currently warm with a temperature '
 'of 26°C. There is a gentle breeze blowing at 4.6 kph. The humidity level is '
 'quite high at 89%, and there is light rain with 0.5 mm of precipitation and '
 'a 92% probability of further rain.')


### Memory and retrieval

The following is a toy example of an agent that reads and writes to an external data source. One characteristic of retrieval systems is that the entire process is **stateless**, e.g. it cannot learn from interactions. Retrieval systems generally involve queries to an external data source, then adding the response to the current model context.

For the following example, the LLM also writes to the same memory store hence affecting future generation states. Hence, it is **stateful**. We can think of the external memory store as the **long-term memory** of the system. On the other hand, LLMs naturally have **short-term memory** in the form of its context. The architecture is shown in @fig-retrieval-system.

![The LLM expresses the intent to write to the memory store via the structured output. Then, it is up to the main program to perform the actual writing. This allows hooks like [guardrails](https://cookbook.openai.com/examples/how_to_use_guardrails) to be applied before executing the function. Note that the retrieval happens prior to LLM processing. It would be nice to have the LLM read the entire filestore but this becomes more expensive as the memory store grows. In practice, information retrieval techniques such as TF-IDF and embedding similarity can be used.
](../img/retrieval-system.png){#fig-retrieval-system}

In [15]:
import json
from typing import List
from openai import OpenAI
from pydantic import BaseModel


class MemoryItem(BaseModel):
    tag: str
    info: str
    reason: str

class RetainResponse(BaseModel):
    items: List[MemoryItem]


class MemoryStore:
    def __init__(self, path="memory.json"):
        """Load memory from JSON file in local path."""
        self.path = path
        self.data = []
        self.tags = set()
        self.load()

    def load(self):
        try:
            self.data = json.load(open(self.path))
        except FileNotFoundError:
            self.reset()

    def save(self):
        with open(self.path, "w") as f:
            json.dump(self.data, f, indent=2)

    def reset(self):
        self.data = []
        self.tags = set()
        self.save()
    
    def add(self, item: dict[str, str]):
        tag = list(item.keys())[0]
        self.tags.add(tag)
        self.data.append(item)

    def __len__(self):
        return len(self.data)
    
    def retrieve(self, query: str, topk: int = 3) -> List[dict]:
        """Simple keyword-based retrieval."""
        
        query_words = query.lower().split()
        retrieved = []
        
        for memory in reversed(self.data):  # <1>
            for tag, info in memory.items():
                memory_text = f"{tag} {info}".lower()

                if any(word in memory_text for word in query_words): # <2>
                    retrieved.append(memory)
            
            if len(retrieved) == topk:
                break
        
        return retrieved


client = OpenAI()
mem = MemoryStore()

1. More recent = more relevant.
2. Substring check. e.g. `'commute' in 'commute_experience'` evaluates to `True`.

Next, we define the LLM generation and write steps:

In [ ]:
def llm_generate(user_input, topk: int = 3):
    """Generate memory items from current input and existing memory."""

    retrieved_memories = mem.retrieve(user_input, topk)
    current_tags = list(mem.tags)
    system_prompt = f"""
You are an assistant that logs daily interactions. 
Relevant past memories based on your current input:
{retrieved_memories}

Current tags:
{current_tags}

Your goal is to identify and retain atomic, standalone facts 
that are likely to be relevant for future interactions.

**GUIDELINES:**

1. **EXTRACT ATOMIC FACTS:**
    - Break down information into the smallest meaningful, self-contained units.
    - **Good**: "User's favorite programmer is Jon Blow."
    - **Bad**: "User mentioned their favorite programmer is Jon Blow who is a famous game programmer" (This has two facts.)
    - The `info` must be a concise, direct paraphrase of the fact. Remove conversational fluff.
    - **Good Info:** "User's favorite city is Tokyo"
    - **Bad Info:** "The user stated that if they had to pick a favorite city, they think it would be Tokyo."

2.  **TAG EFFECTIVELY:**
    - **Format:** Prefer generic, descriptive tags in `snake_case`.
    - **Simple:** Prefer simple tags. Choose `commute` is better than `commute_experience`.
    - **Reuse:** Strongly prefer existing tags. Create a new tag only if necessary.
    - An example: For "I really enjoy hiking in the Alps every summer," a good tag is `hobby` or `outdoor_activity`.
    
4.  **EVALUATE & DECIDE:**
    - It is acceptable to save zero logs from an input if nothing is meaningfully new or relevant.
    - Save multiple logs if the user provides multiple distinct pieces of information.
    - Each memory item should make sense on its own. There should be no dependence between separate logs.
"""
    
    completion = client.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        response_format=RetainResponse,
    )

    return completion.choices[0].message.parsed


def write(user_input: str) -> RetainResponse:
    """Write generated items to memory store."""
    response = llm_generate(user_input)
    for item in response.items:
        mem.add({item.tag: item.info})
    
    mem.save()
    return response

Examples:

In [17]:
import pandas as pd

logs = [
    "Woke up, showered, and left for work at the usual time. Had toast for breakfast before heading out.",
    "Traffic was smooth, arrived at work earlier than usual.",
    "Took the train, had to stand the whole ride since it was packed.",
    "Stopped by the bakery on the way and picked up bread for the team.",
    "Opened my email first thing at the office, mostly routine messages.",
    "Woah! Some guy just came out of nowhere and darted into traffic. That was pretty shocking. Crazy.",
    "Listened to a podcast while walking to the subway.",
    "Grabbed a pen from my desk drawer because mine ran out of ink.",
]

items = []
for input_text in logs:
    for item in write(input_text).items:
        d = item.model_dump()
        d["text"] = input_text
        items.append(d)

df_resp = pd.DataFrame(items)
mem.reset()

The agent decides whether to reuse a tag or create a new one based on the data:

In [20]:
#| code-fold: true
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_colwidth', None)

print("tags:", f"({len(logs)} total logs, {len(df_resp.tag.unique())} tags, {len(df_resp)} saved)")
pprint(list(df_resp.tag.unique()))
df_resp

tags: (8 total logs, 3 tags, 7 saved)
['morning_routine', 'commute', 'work_tool']


,tag,info,reason,text
0,morning_routine,User ate toast for breakfast.,Information about breakfast can be relevant to dietary preferences or habits.,"Woke up, showered, and left for work at the usual time. Had toast for breakfast before heading out."
1,commute,User arrived at work earlier than usual due to smooth traffic.,The fact that traffic conditions affected the user's commute is an atomic detail relevant for potential future interactions about commuting patterns.,"Traffic was smooth, arrived at work earlier than usual."
2,commute,User took a crowded train and had to stand for the whole ride.,"This gives specific details about their commuting experience, which might affect future choices or preferences.","Took the train, had to stand the whole ride since it was packed."
3,commute,User stopped by the bakery on the way to pick up bread.,This is a separate action performed during the commute and complements the previously logged train commute information.,Stopped by the bakery on the way and picked up bread for the team.
4,commute,User witnessed a person darting into traffic.,This standalone fact about a commute experience is unique and might be relevant for understanding similar future conversation contexts.,Woah! Some guy just came out of nowhere and darted into traffic. That was pretty shocking. Crazy.
5,commute,User listens to a podcast while walking to the subway.,This information adds detail to the user's commute routine. It might be relevant for future recommendations or interactions about podcasts.,Listened to a podcast while walking to the subway.
6,work_tool,User keeps extra pens in their desk drawer.,This detail about where extra pens are kept might be useful for future reminders about their workspace or work routine.,Grabbed a pen from my desk drawer because mine ran out of ink.
